In [1]:
# connect to Blob Storage
storage_account_name = "exelonjea"
storage_account_access_key = "otXmOdikFoFztQ5rmK9KD4GqWY5nq4kmEdJ/TGthv5rWR3sXwIcBzT9Cf9IuM6STMTXgk8AgySArv6nsR1fD8w=="
spark.conf.set(
  "fs.azure.account.key." + storage_account_name + ".blob.core.windows.net",
  storage_account_access_key)

In [2]:
# define input and output
input_files = "wasbs://geojson@" + storage_account_name + ".blob.core.windows.net/dc-osm-*.geojson"
output_file = "wasbs://geojson@" + storage_account_name + ".blob.core.windows.net/dc-osm-output.csv"

In [3]:
from pyspark.sql.types import StructType, StructField, StringType

# define schema
sc = StructType([
  StructField("type", StringType(), False),
  StructField("geometry", StringType(), False),
  StructField("properties", StructType([
    StructField("addr:housenumber", StringType()),
    StructField("addr:street", StringType()),
    StructField("addr:city", StringType()),
    StructField("addr:state", StringType()),
    StructField("dcgis:address", StringType()),
    StructField("building:height", StringType()),
    StructField("building:levels", StringType())
  ]), False)
])

# read the JSON data
input = spark.read.option("inferSchema", "false").option("multiLine", "true").option("mode", "DROPMALFORMED").json(input_files, sc)

In [4]:
# should be just over 58k rows (58,315 on the first pull)
input.count()

Out[ 8 ]: 58315

In [5]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, lit
from shapely.geometry import GeometryCollection, shape
import ast

def shape_func(geometry):
  try:
    geodict = ast.literal_eval(geometry)
    geocol = GeometryCollection([shape(geodict).buffer(0)])
    return (geocol.centroid.y, geocol.centroid.x, geocol.area * 1000 * 1000 * 10000)
  except:
    return [None, None, None]

shape_schema = StructType([
  StructField("latitude", FloatType(), True),
  StructField("longitude", FloatType(), True),
  StructField("area", FloatType(), True)
])
shape_udf = udf(shape_func, shape_schema)

def street_func(housenumber, street, address):
  try:
    if housenumber is not None and street is not None: return housenumber + " " + street
    if address is not None: return address
    if street is not None: return street
    return None
  except:
    return None
street_udf = udf(street_func, StringType())

def default_func(value, default):
  try:
    if value is not None: return value
    return default
  except:
    return None
default_udf = udf(default_func, StringType())
  
# transform
output = input.select(
  street_udf("properties.addr:housenumber", "properties.addr:street", "properties.dcgis:address").alias("address"),
  default_udf("properties.addr:city", lit("Washington")).alias("city"),
  default_udf("properties.addr:state", lit("DC")).alias("state"),
  shape_udf("geometry").alias("shape"),
  col("properties.building:height").alias("height"),
  col("properties.building:levels").alias("levels")
).select(
  "address", "city", "state", "shape.latitude", "shape.longitude", "shape.area", "height", "levels"
)


In [6]:
# preview output
output.show()

+--------------------+----------+-----+---------+----------+----------+--------+------+
 address| city|state| latitude| longitude| area| height|levels|
+--------------------+----------+-----+---------+----------+----------+--------+------+
1500 Pennsylvania...|Washington| DC|38.897575| -77.03432| 9635.684| null| 5|
 1 1st St, NE|Washington| DC| 38.89061| -77.00441| 8448.725| 28.0416| 5|
200 New Jersey Av...|Washington| DC|38.886826|-77.006874| 11409.017| null| 5|
400 7th Street So...|Washington| DC|38.884045| -77.02088| 16248.65| 42.672| 10|
1000 Independence...|Washington| DC| 38.8867| -77.02539| 14727.949| null| 8|
 null|Washington| DC|38.886757| -77.02994| 24439.252|21.76272| 6|
333 Constitution ...|Washington| DC|38.892944| -77.01615| 12484.249| null| 8|
 null|Washington| DC| 38.89304|-77.018364| 3781.9465| null| 6|
 null|Washington| DC|38.901558| -77.02304| 10829.877| 57.3024| 15|
 null|Washington| DC|38.899185| -77.02108|10079.9795| null| 12|
935 Pennsylvania ...|Washington| DC|38.895317|-77.024956| 19217.559| null| 9|
 null|Washington| DC| 38.89667| -77.02244| 4445.835| null| 4|
 null|Washington| DC| 38.89782| -77.02997| 2361.3975| null| 12|
1200 Clifton Stre...|Washington| DC|38.921856| -77.0284| 8383.651| null| 6|
 null|Washington| DC| 38.93639| -77.03759| 7261.445| null| 5|
1849 C Street Nor...|Washington| DC|38.894444| -77.04261| 16304.991|28.98648| 8|
1800 F Street Nor...|Washington| DC| 38.8967| -77.04258| 10455.434| null| 6|
1776 G Street Nor...|Washington| DC|38.897938| -77.04121| 3077.4744| null| 8|
500 17th Street N...|Washington| DC|38.895817| -77.04003| 4715.0674| null| 3|
1900 F Street Nor...|Washington| DC| 38.89702|-77.043915| 2079.9373| null| 9|
+--------------------+----------+-----+---------+----------+----------+--------+------+
only showing top 20 rows

In [7]:
# write out a csv
output.write.format("com.databricks.spark.csv").option("header", "true").mode("overwrite").save(output_file)